In [ ]:
import os
import pandas as pd
import numpy as np
import xarray as xr

# On Docker turn off the warning on PROJ by specifying the PROJ lib path (uncomment the following line)
os.environ['PROJ_LIB'] = '/opt/conda/envs/gospl/share/proj'

import meshplex
from scipy.interpolate import interp1d
from scipy.ndimage import gaussian_filter1d

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from mpas_tools.mesh.interpolation import interp_bilin
from perlin_noise import PerlinNoise

import uxarray as uxr
import meshio

from scripts import umeshFcts as ufcts

import matplotlib.pyplot as plt
%matplotlib inline


# Create the mesh

In [ ]:
length = 200e3  # in meter
width = 100e3  # in meter
grid_resolution = 1000.  # in meter
node_x = int(length/grid_resolution+1)
node_y = int(width/grid_resolution+1)
node_x, node_y

In [ ]:
# Build initial surface profile based on given points
xdata = np.array([0,50,60,100,200])*1000
elev = np.array([300,40,10,0,-500])

f = interp1d(xdata, elev)
xnew = np.linspace(0, length, node_x)
ynew = np.linspace(0, width, node_y)
elev_new = f(xnew)
elev_new_gs = gaussian_filter1d(elev_new,sigma=10) 

# Visualize the profile
fig = plt.figure(figsize = (6,3))
plt.rc("font", size=12)
ax = fig.add_axes([0.11, 0.18, 0.85, 0.74])
ax.scatter(xdata/1e3, elev) 
ax.plot(xnew/1e3,elev_new)
ax.plot(xnew/1e3, elev_new_gs)
ax.axhline(y=0., color='k', linestyle='--')
ax.set_xlim([0,200])
ax.set_ylim([-500,400]) 
ax.set_xlabel('Distance (km)')
ax.set_ylabel('Elevation (m)')
plt.show()

In [ ]:
elev = np.broadcast_to(elev_new_gs, (node_x, node_x))[:node_y,:]

In [ ]:
noise1 = PerlinNoise(octaves=3)
noise2 = PerlinNoise(octaves=6)
noise3 = PerlinNoise(octaves=12)
noise4 = PerlinNoise(octaves=24)

xpix = 101
ypix = 201
perlin = np.zeros((xpix,ypix))
for i in range(xpix):
    row = []
    for j in range(ypix):
        noise_val = noise1([i/xpix, j/ypix])
        noise_val += 0.5 * noise2([i/xpix, j/ypix])
        noise_val += 0.25 * noise3([i/xpix, j/ypix])
        noise_val += 0.125 * noise4([i/xpix, j/ypix])

        row.append(noise_val)
    # pic.append(row)
    perlin[i,:] = np.asarray(row)

In [ ]:
noise = np.random.normal(0, 0.1, elev.shape)
ds = xr.Dataset({
    'elev': xr.DataArray(
                data   = elev+perlin*1.5,
                dims   = ['y','x'],
                coords = {'x': xnew,'y': ynew},
                ),
        }
    )

anoise = (np.random.random_sample(elev.shape)+0.5)
wdth = (['y','x'],500.*anoise)
ds['cellwidth'] = (['y','x'],750.*(1+(perlin)))
ds['u'] = (['y','x'],-0.05*np.ones(elev.shape))
ds['v'] = (['y','x'],-0.05*np.ones(elev.shape))
# ds.elev.plot()
ds['cellwidth'].plot()

In [ ]:
output_path = "meshinputs0" 
if not os.path.exists(output_path):
    os.makedirs(output_path)

ufcts.planarMesh(ds,output_path,fvtk='planar.vtk',fumpas=True,voro=True)

The mesh (`base2D.nc`) is now stored in the output folder (here named `slope`). 

We will open this file and extract the information used in goSPL:

In [ ]:
# Loading the UGRID file
ufile = output_path+'/base2D.nc'
var_name = 'data2'
ugrid = uxr.open_grid(ufile) 

# Perform the interpolation (bilinear) 
ufcts.inter2UGRID(ds[['elev','u','v']],ugrid,output_path,var_name,type='face',latlon=False)

data_file = [output_path+'/'+var_name+'.nc']
# Get the information related to the mesh: primal and dual mesh
primal_mesh = uxr.open_dataset(ufile, *data_file, use_dual=False)
dual_mesh = uxr.open_dataset(ufile, *data_file, use_dual=True)

# Extract nodes and faces information
ucoords = np.empty((dual_mesh.uxgrid.n_node,3))
ucoords[:,0] = dual_mesh.uxgrid.node_x.values
ucoords[:,1] = dual_mesh.uxgrid.node_y.values
ucoords[:,2] = dual_mesh.uxgrid.node_z.values
ufaces = primal_mesh.uxgrid.node_face_connectivity.values

# Get information about your mesh:
print("Number of nodes: ",len(ucoords)," | number of faces ",len(ufaces))
edge_min = np.round(dual_mesh.uxgrid.edge_node_distances.min().values/1000.+0.,2)
edge_max = np.round(dual_mesh.uxgrid.edge_node_distances.max().values/1000.+0.,2)
edge_mean = np.round(dual_mesh.uxgrid.edge_node_distances.mean().values/1000.+0.,2)
print("edge range (km): min ",edge_min," | max ",edge_max," | mean ",edge_mean)

We now read the created `vtk` file and add the interpolated variables to it:

In [ ]:
mesh = meshio.read(output_path+'/planar.vtk')
vertex = mesh.points
cells = mesh.cells_dict['triangle']
Umesh = meshplex.MeshTri(vertex, cells)
Uarea = Umesh.control_volumes
print('Cell area (km2): ',Uarea.min()*1.e-6,Uarea.max()*1.e-6)

# Define mesh
paleovtk = output_path+"/init.vtk"
vis_mesh = meshio.Mesh(vertex, {"triangle": cells}, 
                       point_data={"elev": primal_mesh.elev.values,
                                   "u": primal_mesh.u.values,
                                   "v": primal_mesh.v.values,
                                   },
                       )

# Write it disk
meshio.write(paleovtk, vis_mesh)
print("Writing VTK input file as {}".format(paleovtk))

### Creating goSPL input

We will now create the inputs for goSPL. We first start by creating the input mesh defining our UGRID structure:

Save the goSPL mesh:

In [ ]:
out_path = "inputs" 
if not os.path.exists(out_path):
    os.makedirs(out_path)
meshname = out_path+"/gospl_mesh"
np.savez_compressed(meshname, v=vertex, c=cells, 
                    z=primal_mesh.elev.data, uv=np.vstack((primal_mesh.u.data,primal_mesh.v.data)).T
                    )

Make a sea-level curve

In [ ]:
def buildCurve(timeExt = None, timeStep = None, seaExt = None,
                   ampExt = None, periodExt = None):
        """
        Curve created which interpolate linearly the averaged values of sea-level
        trends over the specified time period.

        Parameters
        ----------
        variable: timeExt
            Extent of the simulation time: start/end time (in years)

        variable: timeStep
            Discretisation step for time range (in years).

        variable: seaExt
            Sea level value for starting and ending times (in metres)

        variable: ampExt
            Amplitude of the sea level wave for starting and ending times (in metres)

        variable: periodExt
            Period of the sea level wave for starting and ending times (in years)
        """

        dt = float(timeStep)
        so = float(seaExt[0])
        sm = float(seaExt[1])
        to = float(timeExt[0])
        tm = float(timeExt[1])+dt
        Ao = float(ampExt[0])
        Am = float(ampExt[1])
        Po = float(periodExt[0])
        Pm = float(periodExt[1])

        time = np.arange(to,tm,dt,dtype=float)

        # Sea-level
        a0 = (sm - so)/(tm - to)
        b0 = so - a0 * to
        sl = a0 * time + b0
        # Amplitude
        a1 = (Am - Ao)/(tm - to)
        b1 = Ao - a1 * to
        A = a1 * time + b1
        # Period
        a2 = (Pm - Po)/(tm - to)
        b2 = Po - a2 * to
        P = a2 * time + b2

        for t in range(len(time)):
            sl[t] += A[t] * np.cos(2.* np.pi * (time[t] - to) / P[t])

        return time, sl

timeExt = [0.,5.e5]
timeStep = 5.e3
seaExt = [0.,-40]
ampExt = [1,20]
periodExt = [2.e4,1.e5]
time, sl = buildCurve(timeExt,timeStep,seaExt,ampExt,periodExt)

In [ ]:
# Visualize the profile
fig = plt.figure(figsize = (6,3))
plt.rc("font", size=12)
ax = fig.add_axes([0.11, 0.18, 0.85, 0.74])
ax.plot(time/1000,sl)
ax.set_xlabel('Time (ky)')
ax.set_ylabel('Sea-level (m)')
plt.show()

Save it as file for goSPL simulation:

In [ ]:
df = pd.DataFrame({'X':np.around(time, decimals=0),'Y':np.around(sl, decimals=3)})
df.to_csv('inputs/sealevel.csv',columns=['X', 'Y'], sep=' ', index=False ,header=0)

### Alternative approach to jigsaw using stripy

This approach however does not ensure centroidal voronoi tessellation (CVT) sampling...

In [ ]:
import stripy as stripy
from mpas_tools.mesh.interpolation import interp_bilin

xcoords = ds.x.values
ycoords = ds.y.values

vertices_xy = np.array([
    [xcoords.min(), ycoords.min()],
    [xcoords.max(), ycoords.min()],
    [xcoords.max(), ycoords.max()],
    [xcoords.min(), ycoords.max()],
])
vertices_x = vertices_xy.T[0]
vertices_y = vertices_xy.T[1]

In [ ]:
# Build the mesh
# Changing the refinement level will increase the resolution...
refLvl = 9
triangulation = stripy.Triangulation(x=vertices_x, y=vertices_y, permute=True)
refined_triangulation = stripy.Triangulation(x=vertices_x, y=vertices_y, refinement_levels=refLvl, permute=True)
print('number of points in the triangular grid', refined_triangulation.npoints)

In [ ]:
# Interpolate elevation on the triangular mesh
zData = interp_bilin(xcoords, ycoords, ds['elev'].values, refined_triangulation.x, refined_triangulation.y)

In [ ]:
out_path = "inputs" 
if not os.path.exists(out_path):
    os.makedirs(out_path)
    
meshname = out_path+"/gospl_mesh_stripy"
pts = np.zeros((len(zData),3))
pts[:,:2] = refined_triangulation.points
np.savez_compressed(meshname, v=pts, c=refined_triangulation.simplices, z=zData)

In [ ]:
# You could chose to run it inline here but it is usually slower then running it from the terminal
# !mpirun --allow-run-as-root -np 10 python runModel.py -i input-strati-stripy.yml 